In [1]:
import numpy as np
import pandas as pd
import string

In [2]:
nfeatures    = 214
nbits        = 9
v_sup_width  = 120 # number of valence support vectors
a_sup_width  = 155 # number of arousal support vectors
rom_depth    = 1024
entire_width = 1408
a_sup_base_addr = nfeatures
v_alpha_base_addr = 2 * nfeatures
a_alpha_base_addr = 2 * nfeatures + v_sup_width

In [3]:
v_support_df = pd.read_csv("RFE_{nf}_nbits_{nb}_v_support.txt".format(nf=nfeatures, nb=nbits), header=None)
a_support_df = pd.read_csv("RFE_{nf}_nbits_{nb}_a_support.txt".format(nf=nfeatures, nb=nbits), header=None)
v_alpha_df   = pd.read_csv("RFE_{nf}_nbits_{nb}_v_alpha_vector.txt".format(nf=nfeatures, nb=nbits), header=None)
a_alpha_df   = pd.read_csv("RFE_{nf}_nbits_{nb}_a_alpha_vector.txt".format(nf=nfeatures, nb=nbits), header=None)

In [4]:
v_support_memory = [[0]*nfeatures]*v_sup_width
a_support_memory = [[0]*nfeatures]*a_sup_width

for i in range(v_sup_width):
    v_support_memory[i] = v_support_df[i*nfeatures : i*nfeatures+nfeatures].squeeze().tolist()
    
for i in range(a_sup_width):
    a_support_memory[i] = a_support_df[i*nfeatures : i*nfeatures+nfeatures].squeeze().tolist()
    
v_alpha_memory = v_alpha_df.squeeze().tolist()
a_alpha_memory = a_alpha_df.squeeze().tolist()

In [5]:
def binarize_9bit_signed(num):
    bin_rep = bin(num & 0b111111111)[2:]
    return '0'*(9-len(bin_rep)) + bin_rep

In [45]:
v_support_memory_T = np.array(v_support_memory).transpose()
a_support_memory_T = np.array(a_support_memory).transpose()

v_support_memory_binarized = []
a_support_memory_binarized = []
v_alpha_memory_binarized   = []
a_alpha_memory_binarized   = []

for i in range(nfeatures):
    curr_row = ''.join(list(map(binarize_9bit_signed, v_support_memory_T[i][::-1]))) # [::-1] reverses the supports by row in memory!
    v_support_memory_binarized.append(curr_row)
    
for i in range(nfeatures):
    curr_row = ''.join(list(map(binarize_9bit_signed, a_support_memory_T[i][::-1])))
    a_support_memory_binarized.append(curr_row)
    
for i in range(v_sup_width):
    curr_row = binarize_9bit_signed(v_alpha_memory[i])
    v_alpha_memory_binarized.append(curr_row)
    
for i in range(a_sup_width):
    curr_row = binarize_9bit_signed(a_alpha_memory[i])
    a_alpha_memory_binarized.append(curr_row)

In [46]:
entire_memory = []

for i in range(nfeatures):
    curr_v_sup_row = v_support_memory_binarized[i] + '0'*(entire_width - len(v_support_memory_binarized[i]))
    entire_memory.append(curr_v_sup_row)
    
for i in range(nfeatures):
    curr_a_sup_row = a_support_memory_binarized[i] + '0'*(entire_width - len(a_support_memory_binarized[i]))
    entire_memory.append(curr_a_sup_row)
    
for i in range(v_sup_width):
    curr_v_alpha_row = v_alpha_memory_binarized[i] + '0'*(entire_width - len(v_alpha_memory_binarized[i]))
    entire_memory.append(curr_v_alpha_row)
    
for i in range(a_sup_width):
    curr_a_alpha_row = a_alpha_memory_binarized[i] + '0'*(entire_width - len(a_alpha_memory_binarized[i]))
    entire_memory.append(curr_a_alpha_row)
    
for i in range(rom_depth - (2 * nfeatures + v_sup_width + a_sup_width)):
    entire_memory.append('0'*entire_width)

In [48]:
entire_memory_hex0 = []
entire_memory_hex1 = []
entire_memory_hex2 = []
entire_memory_hex3 = []
entire_memory_hex4 = []
entire_memory_hex5 = []
entire_memory_hex6 = []
entire_memory_hex7 = []
entire_memory_hex8 = []
entire_memory_hex9 = []
entire_memory_hex10 = []

for i in range(rom_depth):
    entire_string = hex(int(entire_memory[i], 2))[2:].zfill(entire_width//4)
    entire_memory_hex0.append(entire_string[0*32:1*32])
    entire_memory_hex1.append(entire_string[1*32:2*32])
    entire_memory_hex2.append(entire_string[2*32:3*32])
    entire_memory_hex3.append(entire_string[3*32:4*32])
    entire_memory_hex4.append(entire_string[4*32:5*32])
    entire_memory_hex5.append(entire_string[5*32:6*32])
    entire_memory_hex6.append(entire_string[6*32:7*32])
    entire_memory_hex7.append(entire_string[7*32:8*32])
    entire_memory_hex8.append(entire_string[8*32:9*32])
    entire_memory_hex9.append(entire_string[9*32:10*32])
    entire_memory_hex10.append(entire_string[10*32:11*32])

In [49]:
f = open('tech-tstech28-cache-local/imem0.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex0[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem0.hex', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem1.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex1[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem1.hex', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem2.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex2[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem2.hex', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem3.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex3[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem3.hex', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem4.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex4[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem4.hex', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem5.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex5[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem5.hex', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem6.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex6[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem6.hex', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem7.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex7[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem7.hex', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem8.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex8[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem8.hex', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem9.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex9[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem9.hex', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem10.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex10[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new/imem10.hex', 'w')
f.writelines(write_lines)
f.close()

In [50]:
f = open('tech-tstech28-cache-new/imem0_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex0)))
f.close()

f = open('tech-tstech28-cache-new/imem1_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex1)))
f.close()

f = open('tech-tstech28-cache-new/imem2_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex2)))
f.close()

f = open('tech-tstech28-cache-new/imem3_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex3)))
f.close()

f = open('tech-tstech28-cache-new/imem4_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex4)))
f.close()

f = open('tech-tstech28-cache-new/imem5_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex5)))
f.close()

f = open('tech-tstech28-cache-new/imem6_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex6)))
f.close()

f = open('tech-tstech28-cache-new/imem7_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex7)))
f.close()

f = open('tech-tstech28-cache-new/imem8_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex8)))
f.close()

f = open('tech-tstech28-cache-new/imem9_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex9)))
f.close()

f = open('tech-tstech28-cache-new/imem10_raw.hex', 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex10)))
f.close()

In [51]:
f = open('tech-tstech28-cache-local/imem0.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex0[i] + '\n')
f = open('tech-tstech28-cache-new/imem0.rom.v', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem1.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex1[i] + '\n')
f = open('tech-tstech28-cache-new/imem1.rom.v', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem2.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex2[i] + '\n')
f = open('tech-tstech28-cache-new/imem2.rom.v', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem3.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex3[i] + '\n')
f = open('tech-tstech28-cache-new/imem3.rom.v', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem4.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex4[i] + '\n')
f = open('tech-tstech28-cache-new/imem4.rom.v', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem5.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex5[i] + '\n')
f = open('tech-tstech28-cache-new/imem5.rom.v', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem6.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex6[i] + '\n')
f = open('tech-tstech28-cache-new/imem6.rom.v', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem7.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex7[i] + '\n')
f = open('tech-tstech28-cache-new/imem7.rom.v', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem8.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex8[i] + '\n')
f = open('tech-tstech28-cache-new/imem8.rom.v', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem9.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex9[i] + '\n')
f = open('tech-tstech28-cache-new/imem9.rom.v', 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem10.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex10[i] + '\n')
f = open('tech-tstech28-cache-new/imem10.rom.v', 'w')
f.writelines(write_lines)
f.close()